In [13]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import fftconvolve
import librosa


In [2]:
%config Completer.use_jedi = False

In [ ]:
def remove_noise(audio_signal,noise_signal,std_away,n_grad_freq,n_grad_time,decrease_prop):
    
    
    noise_stft = librosa.stft(noise_signal,n_fft=n_fft,hop_length=hop_length,win_length=win_length)
    noise_stft_db = librosa.core.amplitude_to_db(noise_stft, ref=1.0, amin=1e-20, top_db=80.0)
    mean_noise = np.mean(noise_stft,axis=1)
    std_noise = np.mean(noise_stft,axis=1)
    noise_thresh = mean_noise + std_away * std_noise
    
    audio_stft = librosa.stft(audio_signal,n_fft=n_fft,hop_length=hop_length,win_length=win_length)
    audio_stft_db = librosa.core.amplitude_to_db(audio_stft, ref=1.0, amin=1e-20, top_db=80.0)
    
    noise_thresh_matrix = np.repeat(noise_thresh.reshape(1,-1),audio_stft.shape[1],axis=0).T
    noise_mask = audio_stft > noise_thresh_matrix
    
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    
    smoothing_filter = fftconvolve(noise_mask,smoothing_filter) * decrease_prop
    
    cleaned_audio_real = np.abs(audio_stft_db) * (1-smoothing_filter)
    cleaned_audio_img = np.imag(audio_signal) (1-smoothing_filter)
    
    cleaned_audio_stft = librosa.core.db_to_amplitude(cleaned_audio_real)*np.sign(audio_stft) + 1j * cleaned_audio_img
    
    cleaned_audio = librosa.istft(cleaned_audio_stft,hop_length=hop_length,win_length=win_length)

    return cleaned_audio
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    